In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import time
import pandas as pd
import config

In [2]:
driver = webdriver.Chrome()  # Mac用の記述
driver.implicitly_wait(10)

In [3]:
# SBIネオモバイルのログインページへアクセス
url_login = "https://trade.sbineomobile.co.jp/login"
driver.get(url_login)
time.sleep(3)  # ページに遷移する前に次の処理が実行されないようにするため

In [4]:
# ログインフォームの要素を取得する
username = driver.find_element(By.NAME, "username")
password = driver.find_element(By.NAME, "password")
login_btn = driver.find_element(By.ID, "neo-login-btn")

# 念のためテキストボックスの中身を空にする
username.clear()
password.clear()

# テキストボックスに値を入力する
username.send_keys(config.SBI_USERNAME)
password.send_keys(config.SBI_PASSWORD)

# ログインボタンをクリックする
login_btn.click()
time.sleep(1)

In [5]:
# SBIネオモバイルのポートフォリオ
url_portfolio = "https://trade.sbineomobile.co.jp/account/portfolio"
driver.get(url_portfolio)
time.sleep(3)

In [6]:
# 銘柄一覧の要素を取得する
div_element = driver.find_element(By.CLASS_NAME, "sp-main")

# 無限スクロールが終わるまでスクロールする
last_height = driver.execute_script("return arguments[0].scrollHeight", div_element)
while True:
    driver.execute_script(
        "arguments[0].scrollTo(0, arguments[0].scrollHeight);", div_element
    )
    time.sleep(2)
    new_height = driver.execute_script("return arguments[0].scrollHeight", div_element)
    if new_height == last_height:
        break
    last_height = new_height

In [7]:
# ページのhtmlを取得してパースする
html = driver.page_source.encode("utf-8")
parsed_html = BeautifulSoup(html, "html.parser")

# 全銘柄の現在値〜預り区分をデータフレームのリストにする
table = parsed_html.findAll("table")
list_df_tables = pd.read_html(str(table))

In [11]:
# リストのデータフレームを一つにまとめる
df_all_stock = pd.DataFrame()
for x in list_df_tables:
    row = x.T[1:2]
    df_all_stock = pd.concat([df_all_stock, row], axis=0)

,0,1,2,3,4,5
1,493円 / 0 0%,38株,0株,17.94%,418 円,特定
1,"3,215円 / 0 0%",7株,0株,0%,"3,215 円",特定
1,"3,770円 / 0 0%",3株,0株,43.67%,"2,624 円",特定
1,"3,460円 / 0 0%",29株,0株,-4.84%,"3,636 円",特定
1,"2,650円 / 0 0%",71株,0株,18.62%,"2,234 円",特定
...,...,...,...,...,...,...
1,"2,013円 / 0 0%",7株,0株,-0.4%,"2,021 円",特定
1,884円 / 0 0%,88株,0株,3.15%,857 円,特定
1,469円 / 0 0%,100株,0株,21.19%,387 円,特定
1,"1,352円 / 0 0%",53株,0株,1.81%,"1,328 円",特定
